# Introduction
This is a notebook using machine learning to output the schedule of suppliers.
First import the data in the form of a csv file. The input of the model will be a date and the output will be a list of days corresponding to the schedule of the suppliers and when they will arrive next.

# Imports

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

2023-06-14 06:43:09.045620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 06:43:09.892828: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-14 06:43:09.892930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

# Load Dataset

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
filepath = 'data/supplier_dates.csv'#supplier schedule on remote server
# filepath = '/content/drive/MyDrive/Documents/uni_work/Bangkit2023/batch1/capstone/repo/ml_modeling/supplier_dates.csv'
df = pd.read_csv(filepath, dtype = {'tanggal':str,'supplier':str})
#remove all punctuation from the supplier names


# Data Transformation

Transform the code of suppliers into their tokenized forms using the tokenizer.

In [4]:
#multi_hot encode the suppliers based of the date and if they visited on that date or not
#df['supplier'] = np.array([i[0] for i in supplier_sequence],dtype = np.int64)

multi_hot = pd.get_dummies(df, columns = ['supplier'],prefix = '', prefix_sep = '')
multi_hot = multi_hot.drop_duplicates()
multi_hot = multi_hot.groupby('tanggal').sum()
multi_hot = multi_hot.reset_index()
multi_hot = multi_hot.sort_values('tanggal', ascending= False)
multi_hot.head()

,tanggal,ABA,ABK,ABS,ADR,AI,AJW,ALBACAR,ALFA,ANIS,...,TIARA,TUAH,UB,UMJ,URB,UTJ,VG,WOI,WT,YIP
386,2023-04-01,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
385,2023-03-31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
384,2023-03-30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
383,2023-03-29,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
382,2023-03-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
#swap values of 0 and 1 in the first row of the dataframe (because the first row is the multihot date for supplier visits)
def process_recent(dataframe):
  """
  Swaps the values of 0 and 1 in the first row of the dataframe
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the swapped values of 0 and 1 in the first row of the dataframe
  """
  for i in range(1, len(dataframe.columns)):
    if dataframe.iloc[0,i] == 0:
      dataframe.iloc[0,i] = -1
    else:
      dataframe.iloc[0,i] = 0
  return dataframe
multi_hot = process_recent(multi_hot)
multi_hot

,tanggal,ABA,ABK,ABS,ADR,AI,AJW,ALBACAR,ALFA,ANIS,...,TIARA,TUAH,UB,UMJ,URB,UTJ,VG,WOI,WT,YIP
386,2023-04-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,-1,-1,0,0
385,2023-03-31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
384,2023-03-30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
383,2023-03-29,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
382,2023-03-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-01-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2022-01-06,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-01-05,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2022-01-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#add previous date column
def next_day(dataframe):
  """
  Adds a column containing the previous date
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the previous date
  """
  dataframe['next_data'] = dataframe['tanggal'].shift(1)
  return dataframe
multi_hot = next_day(multi_hot)
multi_hot

,tanggal,ABA,ABK,ABS,ADR,AI,AJW,ALBACAR,ALFA,ANIS,...,TUAH,UB,UMJ,URB,UTJ,VG,WOI,WT,YIP,next_data
386,2023-04-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,0,-1,-1,-1,-1,-1,0,0,None
385,2023-03-31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2023-04-01
384,2023-03-30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2023-03-31
383,2023-03-29,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,2023-03-30
382,2023-03-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2023-03-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-01-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2022-01-08
3,2022-01-06,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-01-07
2,2022-01-05,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,2022-01-06
1,2022-01-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-01-05


In [7]:
#Get the time difference between the current date and the previous date and convert it to days, keep the first row as 0
def time_diff(dataframe):
  """
  Adds a column containing the time difference between the current date and the previous date and convert it to days, keep the first row as 0
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the time difference between the current date and the previous date and convert it to days, keep the first row as 0
  """
  dataframe['time_diff'] = pd.to_datetime(dataframe['next_data']) - pd.to_datetime(dataframe['tanggal'])
  dataframe['time_diff'] = dataframe['time_diff'].dt.days
  dataframe.iloc[0,dataframe.columns.get_loc('time_diff')] = 0
  return dataframe
multi_hot = time_diff(multi_hot)
multi_hot.pop('next_data')
multi_hot.head()

,tanggal,ABA,ABK,ABS,ADR,AI,AJW,ALBACAR,ALFA,ANIS,...,TUAH,UB,UMJ,URB,UTJ,VG,WOI,WT,YIP,time_diff
386,2023-04-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,0,-1,-1,-1,-1,-1,0,0,0.0
385,2023-03-31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
384,2023-03-30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1.0
383,2023-03-29,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1.0
382,2023-03-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1.0


In [8]:
#update each column
def day_until_visit(dataframe):
  """
  Adds a column containing the days till the next visit
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the since last visit limited to a max of 365
  """
  dataframe = dataframe.copy()
  for i in range(1, len(dataframe.columns)-1):
    for j in range(1, len(dataframe)):
      if dataframe.iloc[j,i] == 0 and dataframe.iloc[j - 1,i] == -1:
        dataframe.iloc[j,i] = -1
      elif dataframe.iloc[j,i] == 0:
        dataframe.iloc[j,i] = dataframe.iloc[j - 1,i] + dataframe.iloc[j,dataframe.columns.get_loc('time_diff')]
      else:
        dataframe.iloc[j,i] = 0
  return dataframe
days_diff = day_until_visit(multi_hot)
days_diff.pop('time_diff')
days_diff.head()

,tanggal,ABA,ABK,ABS,ADR,AI,AJW,ALBACAR,ALFA,ANIS,...,TIARA,TUAH,UB,UMJ,URB,UTJ,VG,WOI,WT,YIP
386,2023-04-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,-1,-1,0,0
385,2023-03-31,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,1,1
384,2023-03-30,0,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,2,-1,-1,-1,-1,0,2,2
383,2023-03-29,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,3,0,-1,-1,0,1,3,3
382,2023-03-28,2,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,4,1,-1,-1,0,2,4,4


In [9]:
#turn dataframe from wide to long
def wide_to_long(dataframe):
  """
  Turns the dataframe from wide to long
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the dates in long form
  """
  dataframe = dataframe.copy()
  dataframe = dataframe.melt(id_vars = ['tanggal'], var_name = 'supplier', value_name = 'days_to_next_visit')
  return dataframe
long_data = wide_to_long(days_diff)
long_data

,tanggal,supplier,days_to_next_visit
0,2023-04-01,ABA,-1
1,2023-03-31,ABA,-1
2,2023-03-30,ABA,0
3,2023-03-29,ABA,1
4,2023-03-28,ABA,2
...,...,...,...
39469,2022-01-07,YIP,1
39470,2022-01-06,YIP,2
39471,2022-01-05,YIP,3
39472,2022-01-04,YIP,4


In [10]:
#remove all rows with -1 as the value for days_to_next_visit
def remove_neg(dataframe):
  """
  Removes all rows with -1 as the value for days_to_next_visit
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the dates with no -1 values
  """
  dataframe = dataframe.copy()
  dataframe = dataframe[dataframe['days_to_next_visit'] != -1]
  return dataframe
long_data = remove_neg(long_data)
long_data.head()

,tanggal,supplier,days_to_next_visit
2,2023-03-30,ABA,0
3,2023-03-29,ABA,1
4,2023-03-28,ABA,2
5,2023-03-27,ABA,3
6,2023-03-25,ABA,0


In [11]:
#add a columns to extract the day of month, month, year, day of week, week of year and day of year
def add_date(dataframe):
  """
  Adds columns to extract the date, month, year, day of week, week of year and day of year
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the dates with the added columns
  """
  dataframe = dataframe.copy()
  dataframe['date'] = pd.to_datetime(dataframe['tanggal'])
  dataframe['year'] = dataframe['date'].dt.year
  dataframe['month'] = dataframe['date'].dt.month
  dataframe['day_of_month'] = dataframe['date'].dt.day
  dataframe['day_of_week'] = dataframe['date'].dt.dayofweek
  dataframe['week_of_year'] = dataframe['date'].dt.isocalendar().week
  dataframe['day_of_year'] = dataframe['date'].dt.dayofyear
  return dataframe
day_features_extracted = add_date(long_data)
day_features_extracted.pop('tanggal')
day_features_extracted.head()

,supplier,days_to_next_visit,date,year,month,day_of_month,day_of_week,week_of_year,day_of_year
2,ABA,0,2023-03-30,2023,3,30,3,13,89
3,ABA,1,2023-03-29,2023,3,29,2,13,88
4,ABA,2,2023-03-28,2023,3,28,1,13,87
5,ABA,3,2023-03-27,2023,3,27,0,13,86
6,ABA,0,2023-03-25,2023,3,25,5,12,84


In [12]:
#turn date into days since epoch
def days_since_epoch(dataframe):
  """
  Turns the date into days since epoch
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the dates with the date column converted to days since epoch
  """
  dataframe = dataframe.copy()
  dataframe['date'] = pd.to_datetime(dataframe['date'])
  dataframe['date'] = dataframe['date'].map(dt.datetime.toordinal)
  return dataframe
day_features_extracted = days_since_epoch(day_features_extracted)
day_features_extracted.head()

,supplier,days_to_next_visit,date,year,month,day_of_month,day_of_week,week_of_year,day_of_year
2,ABA,0,738609,2023,3,30,3,13,89
3,ABA,1,738608,2023,3,29,2,13,88
4,ABA,2,738607,2023,3,28,1,13,87
5,ABA,3,738606,2023,3,27,0,13,86
6,ABA,0,738604,2023,3,25,5,12,84


In [13]:
#cyclically encode the day, month, day of week, week of year and day of year
day_columns = ['month', 'day_of_month', 'day_of_week', 'week_of_year', 'day_of_year']

cyclic_features = np.array(day_features_extracted[day_columns], dtype=np.float32)
cyclic_features = np.sin(cyclic_features) + np.cos(cyclic_features)

#replace the columns with the cyclically encoded columns
for i, col in enumerate(day_columns):
  day_features_extracted[col] = cyclic_features[:, i]

day_features_extracted.head()

,supplier,days_to_next_visit,date,year,month,day_of_month,day_of_week,week_of_year,day_of_year
2,ABA,0,738609,2023,-0.848872,-0.833780,-0.848872,1.327614,1.370246
3,ABA,1,738608,2023,-0.848872,-1.411691,0.493151,1.327614,1.034772
4,ABA,2,738607,2023,-0.848872,-0.691700,1.381773,1.327614,-0.252068
5,ABA,3,738606,2023,-0.848872,0.664237,1.000000,1.327614,-1.307157
6,ABA,0,738604,2023,-0.848872,0.858851,-0.675262,0.307281,0.053167


In [14]:
#move the days_to_next_visit column to the end
def move_days_to_next_visit(dataframe):
  """
  Moves the days_to_next_visit column to the end
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the dates with the days_to_next_visit column moved to the end
  """
  dataframe = dataframe.copy()
  dataframe['days_to_next_visit'] = dataframe.pop('days_to_next_visit')
  return dataframe
day_features_extracted = move_days_to_next_visit(day_features_extracted)
day_features_extracted.head()

,supplier,date,year,month,day_of_month,day_of_week,week_of_year,day_of_year,days_to_next_visit
2,ABA,738609,2023,-0.848872,-0.833780,-0.848872,1.327614,1.370246,0
3,ABA,738608,2023,-0.848872,-1.411691,0.493151,1.327614,1.034772,1
4,ABA,738607,2023,-0.848872,-0.691700,1.381773,1.327614,-0.252068,2
5,ABA,738606,2023,-0.848872,0.664237,1.000000,1.327614,-1.307157,3
6,ABA,738604,2023,-0.848872,0.858851,-0.675262,0.307281,0.053167,0


In [15]:
# turn to dataset
string_feature = day_features_extracted['supplier']
numeric_feature = day_features_extracted[['date', 'year', 'month', 'day_of_month', 'day_of_week', 'week_of_year', 'day_of_year']].values
target = day_features_extracted['days_to_next_visit']
dataset = tf.data.Dataset.from_tensor_slices(((string_feature,numeric_feature),target)).shuffle(len(day_features_extracted))

len(day_features_extracted)

2023-06-14 06:43:17.596587: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 06:43:17.608424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 06:43:17.610121: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 06:43:17.612750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild

32676

In [16]:
#split the dataset into train and test
BATCH_SIZE = 256
BUFFER = len(day_features_extracted)

train_split = int(BUFFER*0.8)
val_split = int(BUFFER*0.1)

train_dataset = dataset.take(train_split)
test_dataset = dataset.skip(train_split)
val_dataset = test_dataset.skip(val_split)
test_dataset = test_dataset.take(val_split)

train_dataset = train_dataset.shuffle(BUFFER).batch(BATCH_SIZE).prefetch(1)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(1)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(1)

# Model Architecture

In [17]:
#create stringlookup layer
def create_string_lookup_layer(dataframe, column_name):
  """
  Creates a string lookup layer
  Args:
    dataframe (dataframe): dataframe containing the dates
    column_name (string): name of the column to be encoded

  Returns:
    string_lookup (object): an instance of the StringLookup class
  """
  dataframe = dataframe.copy()
  string_lookup = tf.keras.layers.StringLookup(vocabulary=dataframe[column_name].unique(), mask_token=None)
  return string_lookup

vectorizer = create_string_lookup_layer(day_features_extracted, 'supplier')

In [18]:
#create a model
def create_model(vectorization_layer):
    """
    Creates a model
    Args:
        vectorization_layer (object): an instance of the StringLookup class

    Returns:
        model (object): an instance of the Sequential class
    """
    string_input = tf.keras.Input(shape=(1,), dtype=tf.string)
    numeric_input = tf.keras.Input(shape=(7,), dtype=tf.float32)

    #create an embedding layer for the string input
    emb = vectorization_layer(string_input)
    emb = tf.keras.layers.Embedding(vectorization_layer.vocabulary_size(), output_dim=10)(emb)
    emb = tf.keras.layers.Flatten()(emb)

    #concatenate the embedding layer with the numeric input
    x = tf.keras.layers.Concatenate()([emb, numeric_input])
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    #create the output layer
    output = tf.keras.layers.Dense(1)(x)

    #create the model
    model = tf.keras.Model(inputs=[string_input, numeric_input], outputs=output)

    return model

In [19]:
#create a model
model = create_model(vectorizer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 string_lookup (StringLookup)   (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 1, 10)        1030        ['string_lookup[0][0]']          
                                                                                                  
 flatten (Flatten)              (None, 10)           0           ['embedding[0][0]']              
                                                                                              

In [20]:
#tune the learning rate
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))

model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-8),
              metrics=['mae', 'mse'])

history = model.fit(train_dataset, epochs=100, callbacks=[lr_schedule], verbose=0)


2023-06-14 06:43:19.449531: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2023-06-14 06:43:19.449583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:221] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2023-06-14 06:43:19.449604: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at matmul_op_impl.h:622 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Graph execution error:

Detected at node 'model/dense/MatMul' defined at (most recent call last):
    File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.

# Analysis

In [ ]:
#plot the loss and accuracy for each learning rate
def plot_loss(history):
  """
  Plots the loss and accuracy for each learning rate
  Args:
    history (object): an instance of the History class

  Returns:
    None
  """
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.ylim([0, 10])
  plt.legend()
  plt.grid(True)

for i in range(len(histories)):
    plot_loss(histories[i])
    plt.title('Learning Rate: ' + str(learning_rates[i]))
    plt.show()



In [ ]:
#choose the best learning rate
best_learning_rate = 0.001
best_model = create_model(best_learning_rate)
best_model.fit(train_dataset.batch(1), epochs=100, validation_data=val_dataset.batch(1), verbose=1)


In [ ]:
#plot the loss and accuracy for the best model
plot_loss(best_model.history)
plt.title('Best Model')
plt.show()

In [ ]:
#evaluate the best model on the test dataset
loss, accuracy = best_model.evaluate(test_dataset.batch(1), verbose=1)
print("Accuracy", accuracy)


In [ ]:
#predict the next visit for each supplier
def predict_next_visit(dataframe):
  """
  Predicts the next visit for each supplier
  Args:
    dataframe (dataframe): dataframe containing the dates

  Returns:
    df (dataframe): dataframe containing the predicted next visit for each supplier
  """
  dataframe = dataframe.copy()
  for i in range(1, len(dataframe.columns)):
    dataframe.iloc[-1,i] = best_model.predict(dataframe.iloc[-1,0])
  return dataframe
predicted = predict_next_visit(renamed)
predicted
